In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

In [21]:
df_train = pd.read_csv(r"C:\Users\Ofir\Desktop\קבצים ךפרוייקט\fashion-mnist_train.csv")
df_test = pd.read_csv(r"C:\Users\Ofir\Desktop\קבצים ךפרוייקט\fashion-mnist_test.csv")
df_train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 785 entries, label to pixel784
dtypes: int64(785)
memory usage: 359.3 MB


In [25]:
train_labels = np.array(df_train.label)
test_labels = np.array(df_test.label)

In [26]:
df_train.drop('label',axis=1, inplace=True)
df_test.drop('label',axis=1, inplace=True)

In [27]:
label_name = ["T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot"]
train_data = np.array(df_train).reshape(df_train.shape[0],28,28,1)

### KNN modle

In [7]:
kf = KFold(n_splits=10, random_state=1, shuffle=True)

k_range = list(range(1,20))
k_scores = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, df_train, train_labels, cv=kf, scoring='accuracy')
    k_scores.append(scores.mean())
print(k_scores)

[0.8522500000000001, 0.8502666666666666, 0.8558333333333333, 0.85815, 0.8572666666666666, 0.8574833333333334, 0.8564, 0.8568833333333334, 0.8550833333333333, 0.8548, 0.8525500000000001, 0.8522333333333334, 0.85115, 0.85115, 0.8498333333333333, 0.8494333333333334, 0.8480166666666665, 0.8474499999999999, 0.8465833333333335]


### Random Forest modle 

In [8]:
rf = RandomForestClassifier()
kf = KFold(n_splits=5, random_state=42, shuffle=True)

scores = cross_val_score(rf, df_train, train_labels, cv=kf)
print(scores.mean())

0.8813833333333333


### Voting Classifier

In [12]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [13]:
estimators = []
model1 = LogisticRegression(); estimators.append(("logistic", model1))
model2 = DecisionTreeClassifier(); estimators.append(("cart", model2))
model3 = SVC(); estimators.append(("svm", model3))

In [14]:
hard_voting = VotingClassifier(estimators,voting='hard')
soft_voting = VotingClassifier(estimators,voting='soft')

In [18]:
kfold = model_selection.KFold(n_splits=10, random_state=42,shuffle=True)

In [ ]:
hv_results = model_selection.cross_val_score(hard_voting, df_train, train_labels, cv=kfold)
print(hv_results.mean())

In [ ]:
sh_results = model_selection.cross_val_score(soft_voting, df_train, train_labels, cv=kfold)
print(sh_results.mean())